<a href="https://colab.research.google.com/github/zinseo964/dam_flow/blob/main/usingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, LSTM

In [11]:
from google.colab import drive
drive.mount('/content/drive')

rain_data211 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172151.csv',encoding='cp949')
rain_data101 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172202.csv',encoding='cp949')
rain_data100 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172215.csv',encoding='cp949')
rain_data212 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172236.csv',encoding='cp949')
rain_data90 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172245.csv',encoding='cp949')

# rain_data = pd.merge(rain_data211, rain_data101, how='outer')
from functools import reduce

rain_data = reduce(lambda x,y : pd.merge(x,y, on='날짜', how='outer'),[rain_data211, rain_data101, rain_data100, rain_data212, rain_data90])

rain_col = [i for i in rain_data.columns if '강수' in i]
rain_data[rain_col] = rain_data[rain_col].fillna(0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
dam_flow = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/only_dam_flow.csv',encoding='utf-8')

df = reduce(lambda x,y : pd.merge(x,y, on='날짜', how='outer'),[rain_data, dam_flow])

In [13]:
df['211강수량(mm)'] = df['211강수량(mm)'].astype(float)
df['101강수량(mm)'] = df['101강수량(mm)'].astype(float)
df['100강수량(mm)'] = df['100강수량(mm)'].astype(float)
df['212강수량(mm)'] = df['212강수량(mm)'].astype(float)
df['90강수량(mm)'] = df['90강수량(mm)'].astype(float)

In [14]:
X = df[['날짜', '211강수량(mm)', '101강수량(mm)', '100강수량(mm)', '212강수량(mm)', '90강수량(mm)']]
X['날짜'] = pd.to_datetime(X['날짜'])
X['year'] = X['날짜'].dt.year
X['month'] = X['날짜'].dt.month
X['day'] = X['날짜'].dt.day
X = X.drop('날짜', axis=1)

<ipython-input-14-6537a5937448>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['날짜'] = pd.to_datetime(X['날짜'])


In [15]:
X_for_scaler = X.drop(['year', 'month', 'day'], axis=1)  # year, month, day, hour 열 제외
scaler = MinMaxScaler(feature_range=(0, 1))
X_for_scaler = scaler.fit_transform(X_for_scaler)

In [16]:
X[['211강수량(mm)', '101강수량(mm)', '100강수량(mm)', '212강수량(mm)', '90강수량(mm)']] = X_for_scaler
Y = df[['유입량(㎥/s)']]

print(X)

      211강수량(mm)  101강수량(mm)  100강수량(mm)  212강수량(mm)  90강수량(mm)  year  month  \
0       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
1       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
2       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
3       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
4       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
...          ...         ...         ...         ...        ...   ...    ...   
5473    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5474    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5475    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5476    0.026984    0.037714    0.011414    0.037037   0.001899  2023     12   
5477    0.012302    0.020190    0.029777    0.021351   0.072688  2023     12   

      day  
0       1  
1       2  
2  

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], 1)))
model.add(Dense(8))
model.add(Dense(1))

In [21]:
# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# 모델 학습
model.fit(X, Y, epochs=50, batch_size=10, verbose=2)

Epoch 1/50
548/548 - 7s - loss: 40162.5859 - mse: 40162.5859 - 7s/epoch - 13ms/step
Epoch 2/50
548/548 - 5s - loss: 36651.5938 - mse: 36651.5938 - 5s/epoch - 8ms/step
Epoch 3/50
548/548 - 5s - loss: 33040.8125 - mse: 33040.8125 - 5s/epoch - 10ms/step
Epoch 4/50
548/548 - 4s - loss: 30871.3086 - mse: 30871.3086 - 4s/epoch - 8ms/step
Epoch 5/50
548/548 - 5s - loss: 29569.3633 - mse: 29569.3633 - 5s/epoch - 9ms/step
Epoch 6/50
548/548 - 5s - loss: 27742.2656 - mse: 27742.2656 - 5s/epoch - 9ms/step
Epoch 7/50
548/548 - 4s - loss: 26748.6055 - mse: 26748.6055 - 4s/epoch - 8ms/step
Epoch 8/50
548/548 - 5s - loss: 25904.8027 - mse: 25904.8027 - 5s/epoch - 9ms/step
Epoch 9/50
548/548 - 5s - loss: 25620.4141 - mse: 25620.4141 - 5s/epoch - 9ms/step
Epoch 10/50
548/548 - 4s - loss: 24935.2695 - mse: 24935.2695 - 4s/epoch - 8ms/step
Epoch 11/50
548/548 - 7s - loss: 24573.2930 - mse: 24573.2930 - 7s/epoch - 12ms/step
Epoch 12/50
548/548 - 4s - loss: 24107.6055 - mse: 24107.6055 - 4s/epoch - 8ms/ste

In [23]:
train_score = model.evaluate(X, Y, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (train_score[0], np.sqrt(train_score[0])))

Train Score: 15131.21 MSE (123.01 RMSE)
